In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


In [2]:
def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [3]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[8]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed




In [4]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_1.run import run_objetivo_1
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.run import run_objetivo_2
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_3.run import run_objetivo_3

In [5]:

# EXTRACT IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.cuadro_averias import (
    extract_averias_table
) 
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.informe_tecnico import (
    extract_tecnico_reports_without_hours_last_dates
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.anexo_indisponibilidad import (
    extract_indisponibilidad_anexos
)

In [6]:
# TRANSFORM IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.averias import ( 
    preprocess_df_word_averias
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.informe_tecnico import ( 
    preprocess_df_word_informe_tecnico
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.anexos import ( 
    preprocess_df_word_anexos_indisponibilidad
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_335 import ( 
    preprocess_335
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_380 import ( 
    preprocess_380
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.cuismp_sharepoint import ( 
    preprocess_df_cid_cuismp_sharepoint
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.corte_excel import ( 
    preprocess_corte_excel
)

In [7]:
# MERGE IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.excel_sga.excel_sga import ( 
merge_sga_335_corte_excel_sharepoint_cuismp_sga380
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.cuadro_averias.datos import ( 
merge_word_datos_averias_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.cuadro_averias.telefonia import ( 
merge_word_telefonia_averias_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.informe_tecnico.datos import ( 
merge_word_datos_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.informe_tecnico.telefonia import ( 
merge_word_telefonia_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.anexo_indisponibilidad.datos import ( 
merge_word_datos_anexos_disponibilidad_df_merged_sga
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.anexo_indisponibilidad.telefonia import ( 
merge_word_telefonia_anexos_disponibilidad_df_merged_sga
)


In [8]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_3.o1_anexos_sustentos_paradas_reloj_validator import validate_anexos_indisponibilidad_word
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_3.o1_anexos_sustentos_paradas_reloj_validator import build_failure_messages_validate_anexos_indisponibilidad_word


In [9]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 3 3.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_29-04-2025_12-05-2025_20250513_162739.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_29-04-2025_12-05-2025_20250513_163320.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS 9.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS 10.docx"

In [10]:
df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=2, engine="openpyxl")
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335) 
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)


df_word_datos_anexos_indis =  extract_indisponibilidad_anexos(DIR_WORD_DATOS)
df_word_telefonia_anexos_indis = extract_indisponibilidad_anexos(DIR_WORD_TELEFONIA)



In [11]:
df_word_datos_anexos_indis

,ticket,indisponibilidad_header,indisponibilidad_periodos,indisponibilidad_footer,indisponibilidad_total
0,21814560,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),05/05/2025 15:37:00 hasta el día 06/05/2025 14:00:00,(Total de horas sin acceso a la sede: 22:23 horas),22:23
1,21812559,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),30/04/2025 10:39:00 hasta el día 30/04/2025 13:00:00\n30/04/2025 14:16:00 hasta el día 05/05/2025 13:00:00,(Total de horas sin acceso a la sede: 121:05 horas),121:05
2,21815846,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),07/05/2025 10:09:00 hasta el día 07/05/2025 13:10:00,(Total de horas sin acceso a la sede: 3:01 horas),03:01
3,21812186,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),30/04/2025 09:47:00 hasta el día 01/05/2025 12:30:00,(Total de horas sin acceso a la sede: 26:43 horas),26:43
4,21812186,,,,
5,21815446,,,,
6,21815618,,,,
7,21815671,,,,
8,21815861,,,,


In [12]:

df_process_word = preprocess_df_word_anexos_indisponibilidad(df_word_datos_anexos_indis)
summary = get_dataframe_summary(df_process_word)
summary
df_process_word

,nro_incidencia,indisponibilidad_header,indisponibilidad_periodos,indisponibilidad_footer,indisponibilidad_total
0,21814560,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),05/05/2025 15:37:00 hasta el día 06/05/2025 14:00:00,(Total de horas sin acceso a la sede: 22:23 horas),22:23
1,21812559,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),30/04/2025 10:39:00 hasta el día 30/04/2025 13:00:00\n30/04/2025 14:16:00 hasta el día 05/05/2025 13:00:00,(Total de horas sin acceso a la sede: 121:05 horas),121:05
2,21815846,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),07/05/2025 10:09:00 hasta el día 07/05/2025 13:10:00,(Total de horas sin acceso a la sede: 3:01 horas),03:01
3,21812186,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),30/04/2025 09:47:00 hasta el día 01/05/2025 12:30:00,(Total de horas sin acceso a la sede: 26:43 horas),26:43
4,21812186,,,,
5,21815446,,,,
6,21815618,,,,
7,21815671,,,,
8,21815861,,,,


In [13]:

df_preprocss_excel = preprocess_corte_excel(df_corte_excel)
df_preprocss_sharepoint_cuismp = preprocess_df_cid_cuismp_sharepoint(df_cid_cuismp_sharepoint)
df_preprocss_sga335 = preprocess_335(df_sga_dinamico_335)
df_preprocss_sga380 = preprocess_380(df_sga_dinamico_380)



In [14]:
df_process_word.head(1) 

,nro_incidencia,indisponibilidad_header,indisponibilidad_periodos,indisponibilidad_footer,indisponibilidad_total
0,21814560,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),05/05/2025 15:37:00 hasta el día 06/05/2025 14:00:00,(Total de horas sin acceso a la sede: 22:23 horas),22:23


In [15]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_preprocss_excel, df_preprocss_sga335,
        df_preprocss_sharepoint_cuismp, df_preprocss_sga380,
        'both'
        )
#df_matched_corte_sga335_Sharepoint_cuismp_sga380.head(1)
#df_matched_corte_sga335_Sharepoint_cuismp_sga380[df_matched_corte_sga335_Sharepoint_cuismp_sga380['nro_incidencia']=='21809231']

No clock stops found for incident 21812396
No clock stops found for incident 21813871
No clock stops found for incident 21813973
No clock stops found for incident 21814333
No clock stops found for incident 21814420
No clock stops found for incident 21814421
No clock stops found for incident 21814454
No clock stops found for incident 21814456
No clock stops found for incident 21814457
No clock stops found for incident 21814476
No clock stops found for incident 21814510
No clock stops found for incident 21814582
No clock stops found for incident 21814664
No clock stops found for incident 21814756
No clock stops found for incident 21815112
No clock stops found for incident 21815127
No clock stops found for incident 21815136
Adjusting end time to interruption en for incident 21815138
No clock stops found for incident 21815140
No clock stops found for incident 21815142
No clock stops found for incident 21815151
No clock stops found for incident 21815152
No clock stops found for incident 218

In [16]:
merge_word_dfs_merged_anexos_dato = merge_word_datos_anexos_disponibilidad_df_merged_sga(
    df_matched_corte_sga335_Sharepoint_cuismp_sga380,
    df_process_word,
        )
row = merge_word_dfs_merged_anexos_dato[merge_word_dfs_merged_anexos_dato['nro_incidencia'] == '21812186']
row
#merge_word_dfs_merged_anexos_dato


,nro_incidencia,FECHA Y HORA INICIO,FECHA Y HORA FIN,ESTATUS,SERVICIO,CUISMP_sga_dinamico_335_excel_matched,TIPO CASO,AVERÍA,TIEMPO (HH:MM),COMPONENTE,DF,DIRECCION,OBSERVACIÓN,CID,FIN-INICIO (HH:MM),DETERMINACIÓN DE LA CAUSA,RESPONSABILIDAD,TIPO REPORTE,Duracion entero,Agrupación entero,USUARIO,ESPECIALISTA,CODINCIDENCEPADRE,MASIVO,MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS,TIPO DE INCIDENCIA,TIEMPO INTERRUPCION,INDISPONIBILIDAD,DC + INDISPONIBILIDAD,TIEMPO (HH:MM)_trimed,FIN-INICIO (HH:MM)_trimed,FECHA_Y_HORA_INICIO_fmt,FECHA_Y_HORA_FIN_fmt,duration_diff_corte_sec,diff_corte_sec_hhmm,fin_inicio_hhmm_column_corte_to_minutes,duration_diff_corte_min,extracted_hour,canal_ingreso,filter,codincidencepadre,masivo,hijos,pxav,estado,mesa_atencion,catg,ubigeo,cnoci,departamento,distrito,caso,tipo_caso,nombre_cliente,codigo_cliente,sector,segm,cid,mediotxcid,mediotxide,mediotxsot,mediotx,direccion,tipopy,srv,um,dias_transcurridos,considerar,comentario,otpint,sotpint,fecgensotpint,pausacliente2pint,otpext,sotpext,fecgensotpext,sotwimax,sotlte,crq,cantidadcrq,seqcrq,crqobs,remedy,site,cantidadremedy,seqremedy,remedyobs,sot,usot,feccomsot,sot_comentario,compute_0055,fo,mo,slacomp,desp,desplazamiento,horas_pendiente,horas_sin_update,tipo_incidencia,tipo_servicio,producto,bandeja_cnoc,dpto_actual,fecha_comunicacion_cliente,fecha_generacion,fg_padre,fecha_instalacion,fecha_apertura,mes_apertura,anio_apertura,dia_apertura,semana_del_mes,hora,hl,dpto_origen,dpto_origen_detalle,area_1ra_anotacion,area_que_derivo_al_cnoc,derivadoalcnoc,demora_derivacion,demora_tomar_ticket,usuariotomaticket,proactivo,usuario_actual,usuario_ultima_anotacion,compute_0090,compute_0091,derivacion_del_operador,area_ultima_anotacion,fecha_derivacion_cierre,mes_derivacion_cierre,dia_derivacion_cierre,usuario_cierra,tipo_usuario_cierra,tipificacion_interrupcion,tip_interr_filtrado,interrupcion_inicio,interrupcion_fin,tiempocliente,tiempo_interrupcion,compute_0105,compute_0106,compute_0107,compute_0108,compute_0109,compute_0110,compute_0111,compute_0112,tiempopint,tiempored,tiempoproveedor,tiempopext,cruce,tipificacion_problema,tipificacion_observacion,tipificacion_tipo,tipificacion_responsable,tiempotransitocnoc,it_userid,it_responsable,it_determinacion_de_la_causa,it_medidas_tomadas,it_conclusiones,reinc,hora_sistema,fecha_generacion_truncated,interrupcion_inicio_truncated,interrupcion_fin_truncated,Expected_Inicio_truncated,Expected_Inicio_truncated_fm,interrupcion_fin_truncated_fm,duration_diff_335,duration_diff_335_sec,diff_335_sec_hhmm,duration_diff_335_min,_merge,CUISMP_sharepoint_cid_cuismp,Distrito Fiscal,%Disponibilidad,BW contratado,SEDE,CID NUEVO,Unnamed: 7,Unnamed: 8,Unnamed: 9,sum_paradas,clock_stops,clock_stops_paragraph,clock_stops_paragraph_header,clock_stops_paragraph_periodos,clock_stops_paragraph_footer,indisponibilidad_header,indisponibilidad_periodos,indisponibilidad_footer,indisponibilidad_total
0,21812186,2025-04-29 15:40:00,2025-05-03 16:35:00,Caso Concluído,Red Privada Virtual Full Mesh,22015,SIN SERVICIO,CORTE DE FIBRA OPTICA / COBRE DE ULTIMA MILLA - CASO FORTUITO,70:12,COMPONENTE II,Loreto,"CA PADRE GINER MZ.20 LT.24A ASENTAMIENTO HUMANO REQUENA -5.061901, -73.855648",Se generó ticket para la revisión del servicio de datos de la sede Requena,23364571,96:55,El incoveniente se originó por un corte de fibra óptica a 250 metros desde la sede del cliente ocasionado por terceros,TERCEROS,RECLAMO,70,Mayor a 24h,E755639,TERCEROS - CORTE,21812186,No,"El cliente, el Sr. Geoffrey Perez, reportó interrupción de su servicio de datos en su sede identificada con el CUISMP 22015 y se generó ticket el día 29/04/2025 a las 15:40 horas. Inmediatamente, Claro revisó el enlace, encontrando la pérdida de conectividad con un equipo en nuestro punto de presencia que atiende al cliente. Posteriormente, se gestionó el desplazamiento de un personal técnico hacia la sede del cliente para la revisión de las conexiones. Dicho personal encontró un corte del cable de fibra 

In [17]:
summa_word = get_dataframe_summary(df_process_word)
summa_word

,Data Type,Non Null Count,Null Count,Null Percentage,Unique Values
nro_incidencia,object,9,0,0.00,8
indisponibilidad_header,object,9,0,0.00,2
indisponibilidad_periodos,object,9,0,0.00,5
indisponibilidad_footer,object,9,0,0.00,5
indisponibilidad_total,object,9,0,0.00,5


In [18]:
sum_excel = get_dataframe_summary(df_preprocss_excel)
#sum_excel

In [19]:
row = df_preprocss_excel[df_preprocss_excel['nro_incidencia'] == '21812186']
#row

In [20]:
df_vali = validate_anexos_indisponibilidad_word(merge_word_dfs_merged_anexos_dato)
#df_vali.head(5)
#df_vali[['nro_incidencia','indisponibilidad_extract','expected_indisponibilidad']]
row = df_vali[df_vali['nro_incidencia']=='21812186']
row


,nro_incidencia,FECHA Y HORA INICIO,FECHA Y HORA FIN,ESTATUS,SERVICIO,CUISMP_sga_dinamico_335_excel_matched,TIPO CASO,AVERÍA,TIEMPO (HH:MM),COMPONENTE,DF,DIRECCION,OBSERVACIÓN,CID,FIN-INICIO (HH:MM),DETERMINACIÓN DE LA CAUSA,RESPONSABILIDAD,TIPO REPORTE,Duracion entero,Agrupación entero,USUARIO,ESPECIALISTA,CODINCIDENCEPADRE,MASIVO,MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS,TIPO DE INCIDENCIA,TIEMPO INTERRUPCION,INDISPONIBILIDAD,DC + INDISPONIBILIDAD,TIEMPO (HH:MM)_trimed,FIN-INICIO (HH:MM)_trimed,FECHA_Y_HORA_INICIO_fmt,FECHA_Y_HORA_FIN_fmt,duration_diff_corte_sec,diff_corte_sec_hhmm,fin_inicio_hhmm_column_corte_to_minutes,duration_diff_corte_min,extracted_hour,canal_ingreso,filter,codincidencepadre,masivo,hijos,pxav,estado,mesa_atencion,catg,ubigeo,cnoci,departamento,distrito,caso,tipo_caso,nombre_cliente,codigo_cliente,sector,segm,cid,mediotxcid,mediotxide,mediotxsot,mediotx,direccion,tipopy,srv,um,dias_transcurridos,considerar,comentario,otpint,sotpint,fecgensotpint,pausacliente2pint,otpext,sotpext,fecgensotpext,sotwimax,sotlte,crq,cantidadcrq,seqcrq,crqobs,remedy,site,cantidadremedy,seqremedy,remedyobs,sot,usot,feccomsot,sot_comentario,compute_0055,fo,mo,slacomp,desp,desplazamiento,horas_pendiente,horas_sin_update,tipo_incidencia,tipo_servicio,producto,bandeja_cnoc,dpto_actual,fecha_comunicacion_cliente,fecha_generacion,fg_padre,fecha_instalacion,fecha_apertura,mes_apertura,anio_apertura,dia_apertura,semana_del_mes,hora,hl,dpto_origen,dpto_origen_detalle,area_1ra_anotacion,area_que_derivo_al_cnoc,derivadoalcnoc,demora_derivacion,demora_tomar_ticket,usuariotomaticket,proactivo,usuario_actual,usuario_ultima_anotacion,compute_0090,compute_0091,derivacion_del_operador,area_ultima_anotacion,fecha_derivacion_cierre,mes_derivacion_cierre,dia_derivacion_cierre,usuario_cierra,tipo_usuario_cierra,tipificacion_interrupcion,tip_interr_filtrado,interrupcion_inicio,interrupcion_fin,tiempocliente,tiempo_interrupcion,compute_0105,compute_0106,compute_0107,compute_0108,compute_0109,compute_0110,compute_0111,compute_0112,tiempopint,tiempored,tiempoproveedor,tiempopext,cruce,tipificacion_problema,tipificacion_observacion,tipificacion_tipo,tipificacion_responsable,tiempotransitocnoc,it_userid,it_responsable,it_determinacion_de_la_causa,it_medidas_tomadas,it_conclusiones,reinc,hora_sistema,fecha_generacion_truncated,interrupcion_inicio_truncated,interrupcion_fin_truncated,Expected_Inicio_truncated,Expected_Inicio_truncated_fm,interrupcion_fin_truncated_fm,duration_diff_335,duration_diff_335_sec,diff_335_sec_hhmm,duration_diff_335_min,_merge,CUISMP_sharepoint_cid_cuismp,Distrito Fiscal,%Disponibilidad,BW contratado,SEDE,CID NUEVO,Unnamed: 7,Unnamed: 8,Unnamed: 9,sum_paradas,clock_stops,clock_stops_paragraph,clock_stops_paragraph_header,clock_stops_paragraph_periodos,clock_stops_paragraph_footer,indisponibilidad_header,indisponibilidad_periodos,indisponibilidad_footer,indisponibilidad_total,indisponibilidad_header_match,indisponibilidad_periodos_match,indisponibilidad_total_match,Validation_OK,fail_count
0,21812186,2025-04-29 15:40:00,2025-05-03 16:35:00,Caso Concluído,Red Privada Virtual Full Mesh,22015,SIN SERVICIO,CORTE DE FIBRA OPTICA / COBRE DE ULTIMA MILLA - CASO FORTUITO,70:12,COMPONENTE II,Loreto,"CA PADRE GINER MZ.20 LT.24A ASENTAMIENTO HUMANO REQUENA -5.061901, -73.855648",Se generó ticket para la revisión del servicio de datos de la sede Requena,23364571,96:55,El incoveniente se originó por un corte de fibra óptica a 250 metros desde la sede del cliente ocasionado por terceros,TERCEROS,RECLAMO,70,Mayor a 24h,E755639,TERCEROS - CORTE,21812186,No,"El cliente, el Sr. Geoffrey Perez, reportó interrupción de su servicio de datos en su sede identificada con el CUISMP 22015 y se generó ticket el día 29/04/2025 a las 15:40 horas. Inmediatamente, Claro revisó el enlace, encontrando la pérdida de conectividad con un equipo en nuestro punto de presencia que atiende al cliente. Posteriormente, se gestionó el desplazamiento de un personal técnic

In [21]:
df_mess = build_failure_messages_validate_anexos_indisponibilidad_word(df_vali)
df_mess

,nro_incidencia,mensaje,TIPO REPORTE,objetivo
1,21812186,\n No coincide texto inicio de WORD ANEXOS INDISPONIBILIDAD : \n\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP: \nSe tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s)\n No coincide paradas de reloj de WORD ANEXOS INDISPONIBILIDAD : \n\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n30/04/2025 09:47:00 hasta el día 01/05/2025 12:30:00\n No coincide total horas sin acceso a la sede de WORD ANEXOS INDISPONIBILIDAD : \n \n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n26:43,RECLAMO,3.1
16,21814526,\n No coincide texto inicio de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP: \nSe tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s)\n No coincide paradas de reloj de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n05/05/2025 09:55:00 hasta el día 05/05/2025 11:25:00\n No coincide total horas sin acceso a la sede de WORD ANEXOS INDISPONIBILIDAD : \n nan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n01:30,RECLAMO,3.1
21,21814837,\n No coincide texto inicio de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP: \nSe tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s)\n No coincide paradas de reloj de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n05/05/2025 16:45:00 hasta el día 06/05/2025 08:00:00\n No coincide total horas sin acceso a la sede de WORD ANEXOS INDISPONIBILIDAD : \n nan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n15:15,RECLAMO,3.1
55,21816288,\n No coincide texto inicio de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP: \nSe tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s)\n No coincide paradas de reloj de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n07/05/2025 17:30:00 hasta el día 08/05/2025 08:30:00\n No coincide total horas sin acceso a la sede de WORD ANEXOS INDISPONIBILIDAD : \n nan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n15:00,RECLAMO,3.1
63,21816572,\n No coincide texto inicio de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP: \nSe tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s)\n No coincide paradas de reloj de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n08/05/2025 10:22:00 hasta el día 09/05/2025 08:00:00\n No coincide total horas sin acceso a la sede de WORD ANEXOS INDISPONIBILIDAD : \n nan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n21:38,RECLAMO,3.1
66,21816769,\n No coincide texto inicio de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP: \nSe tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s)\n No coincide paradas de reloj de WORD ANEXOS INDISPONIBILIDAD : \nnan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n08/05/2025 14:00:00 hasta el día 09/05/2025 09:00:00\n No coincide total horas sin acceso a la sede de WORD ANEXOS INDISPONIBILIDAD : \n nan\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n19:00,RECLAMO,3.1
